# Camera tracking blue object

Use the provided video or webcam

In [1]:
!pip install opencv-python tqdm matplotlib numpy ipywidgets
!jupyter nbextension enable --py widgetsnbextension

You should consider upgrading via the 'c:\users\cheva\anaconda3\python.exe -m pip install --upgrade pip' command.


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [2]:
#le mieux avec un mask bleu

import cv2
import numpy as np

# i = 0 pour main webcam aka built in, 1 for first usb port etc
#cap = cv2.VideoCapture(i)



#cap = cv2.VideoCapture('test.mp4')
cap = cv2.VideoCapture('long.mov')

fW = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fH = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
gW = 44
gH = 42
cam_grid_ratio = (gW/fW, gH/fH)

low_blue = np.array([110,50,50])
up_blue = np.array([132,255,255])

while(1):
    
    _, frame = cap.read()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    mask = cv2.inRange(hsv, low_blue, up_blue)
    res = cv2.bitwise_and(frame,frame, mask= mask)
   

    contours, hierarchy = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)[-2:]
    areas = [cv2.contourArea(c) for c in contours]
    if len(areas) < 1:

        # Display the resulting frame
        frame = cv2.resize(frame, (0, 0), None, .5, .5)
        cv2.imshow('frame',frame)
        # If "q" is pressed on the keyboard, exit this loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

                
        continue

    else:
        # Find the largest moving object in the image
        max_index = np.argmax(areas)
                
        cnt = contours[max_index]
            
        epsilon = 0.005*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True)
        hull = cv2.convexHull(cnt, returnPoints=False)
        cv2.drawContours(frame, [approx], -1, (0, 0, 255), 3)
        
        x,y,w,h = cv2.boundingRect(cnt)
        #cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
 
        # Draw circle in the center of the bounding box
        x2 = x + int(w/2)
        y2 = y + int(h/2)
        cv2.circle(frame,(x2,y2),4,(255,255,0),-1)
 
    # Print the centroid coordinates (we'll use the center of the
    # bounding box) on the image
        text = "x: " + str(x2) + ", y: " + str(y2)
        cv2.putText(frame, text, (x2 - 10, y2 - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
        
        text2 = "x: " + str(int(x2*cam_grid_ratio[0])) + ", y: " + str(gW-int(y2*cam_grid_ratio[1]))
        cv2.putText(frame, text2, (x2 - 10, y2 + 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
        
        frame = cv2.resize(frame, (0, 0), None, .5, .5)
        cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()